In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from newspaper import Article
import nltk

# Ensure the required NLTK data is downloaded
nltk.download('punkt')

def fetch_article_summary(link):
    """
    Fetch and summarize the article content from a URL.
    """
    try:
        # Use Newspaper3k with headers
        article = Article(link)
        article.download()
        article.parse()
        article.nlp()
        return article.summary
    except Exception as e:
        print(f"[ERROR] Newspaper3k failed for {link}: {str(e)}. Falling back to BeautifulSoup.")

        # Fallback to BeautifulSoup
        try:
            response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")
            paragraphs = soup.find_all("p")
            content = " ".join([p.get_text() for p in paragraphs])
            return content[:500] + "..." if len(content) > 500 else content
        except Exception as bs_error:
            print(f"[ERROR] BeautifulSoup also failed for {link}: {str(bs_error)}")
            return "No summary available."




def get_market_news(tickers):
    """
    Fetch market news for the current day, capturing all available fields and generating summaries.
    Only processes news items with 'type' set to 'story'.
    """
    all_news = []
    today = datetime.now().date()
    one_day_ago = today - timedelta(days=1)

    for ticker in tickers:
        stock = yf.Ticker(ticker)

        try:
            news = stock.news
            for item in news:
                try:
                    publish_timestamp = item.get('providerPublishTime', 0)
                    publish_date = datetime.fromtimestamp(publish_timestamp).date()

                    # Filter news to include only today's and yesterday's articles
                    if publish_date >= one_day_ago:
                        # Only summarize articles with type 'story'
                        if item.get('type', '').lower() == 'story':
                            link = item.get('link', '')
                            summary = fetch_article_summary(link) if link else "No summary available."

                            news_item = {
                                'ticker': ticker,
                                'title': item.get('title', ''),
                                'publisher': item.get('publisher', ''),
                                'link': link,
                                'publish_date': datetime.fromtimestamp(publish_timestamp),
                                'summary': summary,  # Include the generated summary
                                'type': item.get('type', ''),  # Original type from Yahoo API
                                'related_tickers': ', '.join(item.get('relatedTickers', [])),  # Comma-separated related tickers
                            }
                            all_news.append(news_item)
                        else:
                            print(f"[INFO] Skipping non-story type: {item.get('type', '')}")
                except Exception as news_item_error:
                    print(f"[ERROR] Error processing news item: {news_item_error}")

        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {str(e)}")

    print(f"[INFO] Fetched {len(all_news)} news articles.")
    return pd.DataFrame(all_news)


def save_to_csv(df, output_dir="market_news"):
    """
    Save processed news data to a CSV file locally.
    """
    try:
        if df.empty:
            print("[INFO] No news data to save.")
            return None

        os.makedirs(output_dir, exist_ok=True)
        filename = f"market_news_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        filepath = os.path.join(output_dir, filename)
        df.to_csv(filepath, index=False, encoding="utf-8")
        print(f"[INFO] News data saved locally at: {filepath}")
        return filepath
    except Exception as e:
        print(f"[ERROR] Failed to save CSV: {str(e)}")
        return None


def main():
    """
    Main function for fetching and saving market news locally.
    """
    try:
        # Fetch general market news
        indices = ['^IXIC', '^DJI', '^RUT', '^GSPC']
        market_news = get_market_news(tickers=indices)
        if not market_news.empty:
            market_news['category'] = 'General'  # Add category for general market

        # Fetch tech stock news
        tech_stocks = ['AAPL', 'GOOGL', 'MSFT']
        tech_news = get_market_news(tickers=tech_stocks)
        if not tech_news.empty:
            tech_news['category'] = 'Tech'  # Add category for tech stocks

        # Combine news
        combined_news = pd.concat([market_news, tech_news], ignore_index=True)

        # Save to CSV locally
        if not combined_news.empty:
            save_to_csv(combined_news)
        else:
            print("[INFO] No news data to save.")
    except Exception as e:
        print(f"[ERROR] Error in main function: {e}")


if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BryceDaniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[INFO] Skipping non-story type: VIDEO
[INFO] Skipping non-story type: VIDEO
[ERROR] Newspaper3k failed for https://finance.yahoo.com/news/equities-mostly-rise-markets-analyze-214815922.html: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\BryceDaniel/nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\BryceDaniel\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
***************************